In [1]:
sc

In [14]:
lines = sc.parallelize(["pandas", "I like pandas"])
lines.first()

'pandas'

In [18]:
nums = sc.parallelize([1,2,3,4])
squared = nums.map(lambda x: x*x).collect()
for num in squared:
    print ("%i " % (num))

1 
4 
9 
16 


In [20]:
lines = sc.parallelize(["hello world","hi"])
words = lines.flatMap(lambda line: line.split(" "))
words.first()

'hello'

In [21]:
lines = sc.parallelize(["hello world","hi"])
words = lines.map(lambda line: line.split(" "))
words.first()

['hello', 'world']

In [15]:
lines = sc.textFile("README.md")
pythonLines = lines.filter(lambda line: "Python" in line)
pythonLines.persist()
pythonLines.first()

'* The Python examples require urllib3'

In [5]:
pythonLines.count()

2

In [1]:
from collections import namedtuple
from pprint import pprint

In [7]:
base_rdd = sc.textFile("/Users/tim.wu/Documents/python_code/python_spark_basics/Spark-Essentials/Datasets/washington-dc-crime/wash_dc_crime_incidents_2013.csv")
base_rdd.take(5)

['CCN,REPORTDATETIME,SHIFT,OFFENSE,METHOD,LASTMODIFIEDDATE,BLOCKSITEADDRESS,BLOCKXCOORD,BLOCKYCOORD,WARD,ANC,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,START_DATE,END_DATE',
 '04104147,4/16/2013 12:00:00 AM,MIDNIGHT,HOMICIDE,KNIFE,8/7/2015 8:34:01 AM,1500 - 1599 BLOCK OF 1ST STREET SW,398943,133729,6,6D,FIRST,105,9,,006400 2,006400,Precinct 127,7/27/2004 8:30:00 PM,7/27/2004 8:30:00 PM',
 '05047867,6/5/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,KNIFE,8/7/2015 8:32:22 AM,6500  - 6599 BLOCK OF PINEY BRANCH ROAD NW,397769,144596,4,4B,FOURTH,402,17,,001901 4,001901,Precinct 59,4/15/2005 12:30:00 PM,',
 '07083463,7/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:32:15 AM,1800 - 1810 BLOCK OF COLUMBIA ROAD NW,396275,139402,1,1C,THIRD,303,1,ADAMS MORGAN,004002 1,004002,Precinct 25,7/14/2007 3:00:00 PM,',
 '09172197,4/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:33:35 AM,2322 - 2499 BLOCK OF ONTARIO ROAD NW,396518,139335,1,

In [9]:
no_header_rdd = base_rdd.filter(lambda line: 'REPORTDATETIME' not in line)
no_header_rdd.take(10)

['04104147,4/16/2013 12:00:00 AM,MIDNIGHT,HOMICIDE,KNIFE,8/7/2015 8:34:01 AM,1500 - 1599 BLOCK OF 1ST STREET SW,398943,133729,6,6D,FIRST,105,9,,006400 2,006400,Precinct 127,7/27/2004 8:30:00 PM,7/27/2004 8:30:00 PM',
 '05047867,6/5/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,KNIFE,8/7/2015 8:32:22 AM,6500  - 6599 BLOCK OF PINEY BRANCH ROAD NW,397769,144596,4,4B,FOURTH,402,17,,001901 4,001901,Precinct 59,4/15/2005 12:30:00 PM,',
 '07083463,7/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:32:15 AM,1800 - 1810 BLOCK OF COLUMBIA ROAD NW,396275,139402,1,1C,THIRD,303,1,ADAMS MORGAN,004002 1,004002,Precinct 25,7/14/2007 3:00:00 PM,',
 '09172197,4/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:33:35 AM,2322 - 2499 BLOCK OF ONTARIO ROAD NW,396518,139335,1,1C,THIRD,303,1,,003800 1,003800,Precinct 24,5/22/2009 1:00:00 PM,5/22/2009 3:00:00 AM',
 '09251354,2/27/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:32:39 AM,2500 - 2699 BLOCK OF VIRGINIA AVENUE NW,395232,136881,2,2A,SECOND,

In [26]:
CrimeData = namedtuple('CrimeData', ['ccn', 'report_time', 'shift', 'offense', 'method'])

def map_line(line):
  columns = line.split(",")[:5]
  return CrimeData(*columns)

data_rdd = no_header_rdd.map(map_line)
pprint(data_rdd.take(10))

[CrimeData(ccn='04104147', report_time='4/16/2013 12:00:00 AM', shift='MIDNIGHT', offense='HOMICIDE', method='KNIFE'),
 CrimeData(ccn='05047867', report_time='6/5/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='KNIFE'),
 CrimeData(ccn='07083463', report_time='7/8/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='OTHERS'),
 CrimeData(ccn='09172197', report_time='4/8/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='OTHERS'),
 CrimeData(ccn='09251354', report_time='2/27/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='OTHERS'),
 CrimeData(ccn='10028985', report_time='2/27/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='OTHERS'),
 CrimeData(ccn='10033521', report_time='10/10/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='OTHERS'),
 CrimeData(ccn='10124918', report_time='4/9/2013 12:00:00 AM', shift='MIDNIGHT', offense='SEX ABUSE', method='OTHERS'),
 CrimeData(ccn='11010107', report_time

In [29]:
grouped_by_offense_rdd = data_rdd.groupBy(lambda data: data.offense)

# What does this return? You'll need to know for the next step.
print (grouped_by_offense_rdd.take(10))

[('HOMICIDE', <pyspark.resultiterable.ResultIterable object at 0x10c6a7b70>), ('ASSAULT W/DANGEROUS WEAPON', <pyspark.resultiterable.ResultIterable object at 0x10c6a7898>), ('ROBBERY', <pyspark.resultiterable.ResultIterable object at 0x10c947ba8>), ('BURGLARY', <pyspark.resultiterable.ResultIterable object at 0x10c6a7e48>), ('THEFT F/AUTO', <pyspark.resultiterable.ResultIterable object at 0x10c6a7dd8>), ('MOTOR VEHICLE THEFT', <pyspark.resultiterable.ResultIterable object at 0x10c6ac320>), ('ARSON', <pyspark.resultiterable.ResultIterable object at 0x10c6ac5c0>), ('THEFT/OTHER', <pyspark.resultiterable.ResultIterable object at 0x10c6ac668>), ('SEX ABUSE', <pyspark.resultiterable.ResultIterable object at 0x10da0fcf8>)]


In [30]:
offense_counts = grouped_by_offense_rdd.map(lambda g: (g[0], len(g[1]))).collect()
for offense, count in offense_counts:
  print ("{0:30s} {1:d}".format(offense, count))

HOMICIDE                       104
ASSAULT W/DANGEROUS WEAPON     2309
ROBBERY                        4071
BURGLARY                       3370
THEFT F/AUTO                   10130
MOTOR VEHICLE THEFT            2675
ARSON                          35
THEFT/OTHER                    12904
SEX ABUSE                      299


In [33]:
offense_counts = data_rdd.map(lambda item: (item.offense, item)).countByKey()
for offense, counts in offense_counts.items():
  print ("{0:30s} {1:d}".format(offense, counts))

ASSAULT W/DANGEROUS WEAPON     2309
ROBBERY                        4071
THEFT F/AUTO                   10130
SEX ABUSE                      299
MOTOR VEHICLE THEFT            2675
BURGLARY                       3370
THEFT/OTHER                    12904
HOMICIDE                       104
ARSON                          35


In [34]:
grouped_by_offense_rdd.collectAsMap()

{'ARSON': <pyspark.resultiterable.ResultIterable at 0x10c5f5128>,
 'ASSAULT W/DANGEROUS WEAPON': <pyspark.resultiterable.ResultIterable at 0x10c542ba8>,
 'BURGLARY': <pyspark.resultiterable.ResultIterable at 0x10c5422b0>,
 'HOMICIDE': <pyspark.resultiterable.ResultIterable at 0x10c6c7748>,
 'MOTOR VEHICLE THEFT': <pyspark.resultiterable.ResultIterable at 0x10c5f50f0>,
 'ROBBERY': <pyspark.resultiterable.ResultIterable at 0x10c720b38>,
 'SEX ABUSE': <pyspark.resultiterable.ResultIterable at 0x10db4ba20>,
 'THEFT F/AUTO': <pyspark.resultiterable.ResultIterable at 0x10c542320>,
 'THEFT/OTHER': <pyspark.resultiterable.ResultIterable at 0x10ca6c048>}

In [36]:
# How many partitions does the base RDD have? What about the `grouped_by_offense` RDD? How can you find out?
print (base_rdd.getNumPartitions())
print (grouped_by_offense_rdd.getNumPartitions())

2
2


In [38]:
print ("Count of lines: {0}".format(base_rdd.count()))
totalChars = base_rdd.map(lambda line: len(line)).reduce(lambda a, b: a + b)
print ("Count of characters (Unicode): {0}".format(totalChars))

Count of lines: 35898
Count of characters (Unicode): 7906305


In [39]:
data_rdd.cache()

PythonRDD[50] at RDD at PythonRDD.scala:48

In [40]:
result_rdd1 = data_rdd.filter(lambda item: item.offense == 'HOMICIDE')\
                      .map(lambda item: (item.method, 1))\
                      .reduceByKey(lambda i, j: i + j)

for method, count in result_rdd1.collect():
  print ("{0:10s} {1:d}".format(method, count))

KNIFE      11
GUN        81
OTHERS     12


In [41]:
print (data_rdd.map(lambda item: (item.shift, 1))
               .reduceByKey(lambda c1, c2: c1 + c2)
               .max(key=lambda t: t[1]))

('EVENING', 15167)


In [42]:
# COMMAND ----------

# MAGIC %md ### Demonstration
# MAGIC 
# MAGIC Let's plot murders by month. DataFrames are useful for this one.

# COMMAND ----------

# MAGIC %md To do this properly, we'll need to parse the dates. That will require knowing their format. A quick sampling of the data will help.

# COMMAND ----------

data_rdd.map(lambda item: item.report_time).take(30)

['4/16/2013 12:00:00 AM',
 '6/5/2013 12:00:00 AM',
 '7/8/2013 12:00:00 AM',
 '4/8/2013 12:00:00 AM',
 '2/27/2013 12:00:00 AM',
 '2/27/2013 12:00:00 AM',
 '10/10/2013 12:00:00 AM',
 '4/9/2013 12:00:00 AM',
 '7/31/2013 12:00:00 AM',
 '1/31/2013 12:00:00 AM',
 '7/8/2013 12:00:00 AM',
 '1/9/2013 12:59:00 AM',
 '3/23/2013 10:00:00 AM',
 '8/19/2013 12:00:00 AM',
 '5/13/2013 12:00:00 AM',
 '9/10/2013 6:43:00 PM',
 '11/13/2013 12:00:00 AM',
 '8/26/2013 12:00:00 AM',
 '1/1/2013 12:15:00 AM',
 '1/1/2013 12:50:00 AM',
 '1/1/2013 12:18:00 AM',
 '1/1/2013 2:10:00 AM',
 '1/1/2013 1:28:00 AM',
 '1/1/2013 3:00:00 AM',
 '1/1/2013 2:29:00 AM',
 '1/1/2013 2:24:00 AM',
 '1/1/2013 3:10:00 AM',
 '1/1/2013 3:12:00 AM',
 '1/1/2013 3:31:00 AM',
 '1/1/2013 3:45:00 AM']

In [43]:
# COMMAND ----------

# MAGIC %md Okay. We can now create a [`strptime()` format string](https://docs.python.org/2.7/library/datetime.html?highlight=strptime#datetime.datetime.strptime), allowing us to use the `datetime.datetime` Python class to parse those strings into actual datetime values.

# COMMAND ----------

from datetime import datetime
date_format = "%m/%d/%Y %I:%M:%S %p"

# COMMAND ----------

In [ ]:
# MAGIC %md Now, we can create the data frame. We'll start with the `no_header_rdd` and map it slightly differently than we did to create `data_rdd`:

# COMMAND ----------

from pyspark.sql.types import *
def make_row(line):
  columns = line.split(",")[:5]
  columns[1] = datetime.strptime(columns[1], date_format)
  return CrimeData(
    columns[0],
    columns[1],
    columns[2],
    columns[3],
    columns[4])

df = no_header_rdd.map(make_row).toDF()


# COMMAND ----------

df.printSchema()

# COMMAND ----------

# MAGIC %md Let's use a user-defined function (something supported by DataFrames and Spark SQL) to give us a `month` function we can use to extract just the month part of a `Timestamp`.

# COMMAND ----------

month = udf(lambda dt: dt.month)

# COMMAND ----------

display( 
  df.filter(df['offense'] == 'HOMICIDE')
    .select(month(df['report_time']).alias("month"), df['offense'])
    .groupBy('month').count()
)

# COMMAND ----------

# MAGIC %md What about all crimes per month?

# COMMAND ----------

display(
  df.select(month(df['report_time']).alias('month'))
    .groupBy('month').count()
)

# COMMAND ----------

# MAGIC %md We can also plot the frequency of crimes by hour of day.

# COMMAND ----------

hour = udf(lambda dt: dt.hour)

# COMMAND ----------

display(
  df.select(hour(df["report_time"]).alias("hour"), df["offense"]).groupBy("hour").count()
)

# COMMAND ----------